# 실무예제 3-2

## 다음은 2013년 전라남도 유망중소기업 지정업체 명단의 일부이다. 시군 속성과 지정구분 속성 간의 연관성 여부를 카이제곱 검정 방법에 의해 판단해 보시오. (단, 유의 수준은 0.05이다)

### 데이터 파일 : ch3-2(유망중소기업현황).csv

### 원본 투플수 : 386개

In [2]:
# ch3-2.py
import pandas as pd
import numpy as np
import cx_Oracle      # Oracle DB 연동을 위한 cx_Oracle 패키지 임포트

# 데이터로드 (ch2-2.csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('.jupyter/ch3-2(유망중소기업현황).csv', encoding='CP949', engine='python')

In [3]:
rawData_org

,연번,시군,지정구분,기업명,대표자,소 재 지,주생산품,전화번호(061),비고\n(지정번호)
0,1,목포시,기술유망,브로드컴㈜,이봉헌,목포시 석현동 1175(벤처지원 202),정보통신,284-0017,11-1
1,2,목포시,기술유망,㈜케이에스,김시오,목포시 연산동1236-3,용접철망외,1588-4118,11-2
2,3,목포시,기술유망,삼진물산(주),김관석,목포시 연산동 1239-1,참치통조림,270-6113,11-3
3,4,목포시,기술유망,㈜혜성,전재두,목포시 연산동1237-3,가드레일외,1588-2811,11-4
4,5,목포시,기술유망,(유)한국메이드,이승룡,목포시 연산동 1238-4,선박블록,278-4411,11-5
...,...,...,...,...,...,...,...,...,...
381,382,완도군,지역특화,㈜세영산업,김주효,완도군 완도읍가용리,찐톳.건미역,555-0294,11-276
382,383,완도군,기술유망,동주씨테크㈜,이용철,완도군 완도읍 가용리 1086-6,개량부자 전복양식자재,554-6690,12-63
383,384,진도군,수출유망,대대로영농조합법인,김애란,진도군 군내면 명량대첩로 288-23,홍주,542-3399,11-290
384,385,진도군,지역특화,진도정미소영농조합법인,이운갑,진도군 의신면 돈지리 144,"흑미, 기타유색미",544-3459,12-64


In [4]:
# 원본 데이터파일에서 '시군'과 '지정구분' 속성만 추출
rawData = rawData_org.loc[:,'시군':'지정구분']

In [5]:
rawData

,시군,지정구분
0,목포시,기술유망
1,목포시,기술유망
2,목포시,기술유망
3,목포시,기술유망
4,목포시,기술유망
...,...,...
381,완도군,지역특화
382,완도군,기술유망
383,진도군,수출유망
384,진도군,지역특화


### pandas 데이터프레임에 카이제곱 검정을 위한 관측도수/기대도수 분할표 저장
### ‘시군’ 속성 ‘지정구분’ 속성에 대한 관측도수와 기대도수
#### 관측도수 :  ‘시군’과 ‘지정구분’ 속성값의 조합에 대한 실제 출현횟수
#### 기대도수 : 전체 행의 개수와 각 속성의 카디널러티(cardinality, 서로 다른 속성값의 개수)를 구하여 산출
#### 실제 출현하지 않은 (시군+지정구분) 속성값도 포함시키기 위해서 외부조인(outer join)을 수행

In [6]:
# 관측도수 구하기
obs_df = pd.DataFrame(data=rawData.groupby(['시군','지정구분'])['지정구분'].count())
obs_df.columns=['관측도수']
obs_df.reset_index(inplace=True)
obs_df

,시군,지정구분,관측도수
0,강진군,기술유망,2
1,강진군,수출유망,1
2,강진군,지역특화,4
3,고흥군,기술유망,5
4,고흥군,수출유망,1
5,곡성군,기술유망,5
6,곡성군,수출유망,1
7,광양시,기술유망,37
8,광양시,수출유망,7
9,광양시,지역특화,6


In [7]:
# 기대도수 구하기
# 시군별 개수(도수)
sigun_df = pd.DataFrame(data=rawData.groupby(['시군'])['지정구분'].count())
sigun_df.columns=['기대도수_시군']
sigun_df.reset_index(inplace=True)
sigun_df

,시군,기대도수_시군
0,강진군,7
1,고흥군,6
2,곡성군,6
3,광양시,50
4,구례군,8
5,나주시,35
6,담양군,17
7,목포시,20
8,무안군,17
9,보성군,12


In [8]:
# 지정구분별 개수(도수)
gubun_df = pd.DataFrame(data=rawData.groupby(['지정구분'])['지정구분'].count())
gubun_df.columns=['기대도수_지정구분']
gubun_df.reset_index(inplace=True)
gubun_df

,지정구분,기대도수_지정구분
0,기술유망,252
1,수출유망,61
2,지역특화,73


In [9]:
# 원본 데이터프레임의 전체 행의 개수
tot_rows = rawData.shape[0]
tot_rows

386

In [10]:
import itertools as its

# 두 데이터프레임 간 cartesian product(곱집합) 구하기 (리스트로 변환 후 곱집합 연산 수행)
list_pro = list(its.product(sigun_df.values.tolist(), gubun_df.values.tolist()))

# 곱집합 결과 리스트를 저장하는 변수
list_flat = []

# 기대도수를 저장하는 리스트 변수
exp_list = []

# list_pro의 각 행에 대해서 2차원구조를 1차원으로 flatting
for list_ele in list_pro :
    list_flat.append(np.array(list_ele).flatten().tolist())

# list_flat 각 행에 대해서 기대도수를 구함    
for list_ele in list_flat :
    exp_list.append(float(list_ele[1])*float(list_ele[3])/tot_rows)

In [11]:
# 기대도수 구하기
exp_df = pd.DataFrame(data={'시군':np.array(list_flat).T[0],
                            '지정구분':np.array(list_flat).T[2],
                            '기대도수':exp_list})    
exp_df

,시군,지정구분,기대도수
0,강진군,기술유망,4.569948
1,강진군,수출유망,1.106218
2,강진군,지역특화,1.323834
3,고흥군,기술유망,3.917098
4,고흥군,수출유망,0.948187
...,...,...,...
61,해남군,수출유망,1.106218
62,해남군,지역특화,1.323834
63,화순군,기술유망,20.238342
64,화순군,수출유망,4.898964


In [12]:
# 관측도수데이터프레임(obs_df)와 기대도수데이터프레임(exp_df)를 merge(조인)하여 1차원 분할표를 구하기
count_df = pd.merge(obs_df, exp_df, on=['시군','지정구분'], how='right')

# 외부조인에 의해 발생한 결측값을 0으로 채우기
count_df.fillna(0, inplace=True)

In [13]:
# 관측도수/기대도수 분할표 출력
count_df

,시군,지정구분,관측도수,기대도수
0,강진군,기술유망,2.0,4.569948
1,강진군,수출유망,1.0,1.106218
2,강진군,지역특화,4.0,1.323834
3,고흥군,기술유망,5.0,3.917098
4,고흥군,수출유망,1.0,0.948187
...,...,...,...,...
61,해남군,수출유망,1.0,1.106218
62,해남군,지역특화,2.0,1.323834
63,화순군,기술유망,23.0,20.238342
64,화순군,수출유망,3.0,4.898964


In [14]:
# 2개 속성에 대한 자유도(degree of freedom) 갭 구하기
# A 속성에 대한 cardinality = a, B 속성에 대한 cardinality = b라 가정
# cardinality : 서로 다른 속성값의 개수

# 분할표 전체 행 갯수(a*b) 구하기
tot_rows = count_df.shape[0]

### ‘시군’ 속성의 카디널러티(a) = 22
### ‘지정구분’ 속성의 카디널러티(b) = 3
### 그래서, (시군+지정구분) 속성조합의 카디널러티(a\*b) = 22 * 3 = 66

In [15]:
tot_rows  # 변수확인

66

In [16]:
# '시군' 속성의 cardinality 구하기
card_sigun = len(count_df['시군'].unique())

# '지정구분' 속성의 cardinality 구하기
card_gubun = len(count_df['지정구분'].unique())

In [17]:
# cardinality 갭 [(a*b-1) - (a-1)*(b-1)] 구하기
v_ddof = (tot_rows - 1) - (card_sigun-1)*(card_gubun-1)

### 속성조합의 자유도(degree of freedom) = (66-1) = 65
### 실제 자유도는 (22-1) * (3-1) = 42
### 위 두 값의 차이 : 23 -> chisquare() 함수의 세번째 패러미터로 사용

In [18]:
v_ddof  # 변수확인

23

In [19]:
obs_array = count_df['관측도수'] # obs_array : 관측도수를 저장하는 1차원 배열

### obs_array : chisquare() 함수의 첫번째 패러미터로 사용

In [20]:
obs_array  # 변수확인

0      2.0
1      1.0
2      4.0
3      5.0
4      1.0
      ... 
61     1.0
62     2.0
63    23.0
64     3.0
65     5.0
Name: 관측도수, Length: 66, dtype: float64

In [21]:
exp_array = count_df['기대도수'] # exp_array : 기대도수를 저장하는 1차원 배열

### exp_array : chisquare() 함수의 두번째 패러미터로 사용

In [22]:
exp_array  # 변수확인

0      4.569948
1      1.106218
2      1.323834
3      3.917098
4      0.948187
        ...    
61     1.106218
62     1.323834
63    20.238342
64     4.898964
65     5.862694
Name: 기대도수, Length: 66, dtype: float64

In [23]:
# 카이제곱검정을 위한 scipy 패키지 중 stats 모듈 임포트
from scipy import stats

In [24]:
# stats.chisquare() : 카이제곱검정 함수
chis = stats.chisquare(obs_array, exp_array, ddof=v_ddof)

### chisquare() 함수 : 카이제곱(χ2) 통계량을 산출하는 함수
#### 첫번째 패러미터 : 1차원 배열 형태의 관측도수 리스트
#### 두번째 패러미터 : 1차원 배열 형태의 기대도수 리스트
#### 세번째 패러미터 : 자유도 차이 [(a\*b-1) - (a-1)\*(b-1)]

In [25]:
# stats.chisquare() 수행 후의 카이제곱 통계량과 p-value
print("statistic = %.3f, p-value = %.5f" % (chis))

statistic = 54.544, p-value = 0.09287


### 카이제곱 통계량(statistic) : 54.544
### 유의확률(p-value) : 0.09287
### 유의확률이 유의수준 0.05로 설정한다면 귀무가설(두 속성은 연관성이 없다)이 채택됨. 결론적으로, 시군과 지정구분 사이에는 큰 연관성은 없는 것으로 판단할 수 있음.